# Settings

### Autoreload of ipynb

In [250]:
%reload_ext autoreload
%autoreload 2

### Changing the path to the main directories path

In [251]:
import os
os.chdir(r"C:\Users\amilion\Documents\GitHub\CNRL-Cortical-Column\src")
os.getcwd()

'C:\\Users\\amilion\\Documents\\GitHub\\CNRL-Cortical-Column\\src'

# Imports

In [252]:
from conex import *
from pymonntorch import *

from L56.RefrenceFrames import RefrenceFrame
from L56.stimuli.current_base import RandomInputCurrent
from InputLayer.DataLoaderLayer import DataLoaderLayer
from InputLayer.synapse.LocationCoder import LocationCoder
from L423.network.SetTarget import SetTarget

from L56.tools.visualization import refrence_frame_raster



# Configs

In [253]:
HIGHER_LAYER_DIM = 16 
REFRENCE_FRAME_DIM = 23
REFRENCE_INH_DIM = 15
ITERATIONS = 100
SCREEN_SHOT_PATH = "C:\\Users\\amilion\\Desktop\\develop\\python\\NS\\records\\L5.6"

# Defining emty DataLoader

In [254]:
class EmptyDataLoader():
    def __init__(self):
        self.dataset = torch.tensor([0]).reshape(1, 1, 1)
        super().__init__()

dl = EmptyDataLoader()
dl.dataset

tensor([[[0]]])

# Defining targets

In [255]:
target = torch.Tensor([1])

# Initializing the Network

In [256]:
net = Neocortex(index=True, dt=1, dtype=torch.float32, behavior = prioritize_behaviors(
    [
        Payoff(initial_payoff = 1),
        Dopamine(tau_dopamine = 5),
    ]
    ) | {5 : SetTarget(target = target), 601 : Recorder(["dopamine"])})

# Neuron Groups

### Defining L23

In [257]:
higher_layer = NeuronGroup(
    net=net,
    size=NeuronDimension(depth=1, height=HIGHER_LAYER_DIM, width=HIGHER_LAYER_DIM),
    behavior=prioritize_behaviors(
        [
            SimpleDendriteStructure(),
            SimpleDendriteComputation(),
            NeuronAxon(),
        ]
    )
    | (
        {
            250: RandomInputCurrent(prob_to_spike=0.5, T=10),
            # 600: Recorder(["I", "v"]),
            603: EventRecorder(["spikes"]),
        }
    ),
)

L23 = Layer(
    net=net, 
    neurongroups=[higher_layer], 
    input_ports={"input": (None, [Port(object=higher_layer, label=None)])}, 
    output_ports={"output":(None, [Port(object=higher_layer, label=None)])}
)

### Defining L56

In [258]:
L56 = RefrenceFrame(
    net=net, 
    k=2, 
    refrence_frame_side=REFRENCE_FRAME_DIM, 
    inhibitory_size=REFRENCE_INH_DIM,
    competize=True).build_layer()

### Defining InputLayer

In [259]:
InputLayer = DataLoaderLayer(
    net=net, 
    data_loader=dl, 
    widnow_size=1, 
    saccades_on_each_image=1, 
    rest_interval=1, 
    iterations=ITERATIONS).build_data_loader()

# Synapses

In [260]:
Inp_to_L56 = Synapsis(
    net = net,
    src = InputLayer,
    dst = L56,
    input_port = "data_out",
    output_port = "input",
    synapsis_behavior=prioritize_behaviors([
        SynapseInit(),]) | {
        275: LocationCoder()
    },
    synaptic_tag="Proximal"
)

In [261]:
L23_to_L56 = Synapsis(
    net=net,
    src=L23,
    dst=L56,
    input_port="output",
    output_port="input",
    synaptic_tag="Apical, exi",
    synapsis_behavior=prioritize_behaviors(
        [SynapseInit(), SimpleDendriticInput() ,WeightInitializer(mode="normal(10, 0.2)")]
    )
)

L56_to_L23 = Synapsis(
    net=net,
    src=L56,
    dst=L23,
    input_port="output",
    output_port="input",
    synaptic_tag="Apical, exi",
    synapsis_behavior=prioritize_behaviors(
        [SynapseInit(), SimpleDendriticInput() ,WeightInitializer(mode="normal(10, 0.2)")]
    )
)

# Simulating the network

In [262]:
net.initialize()
net.simulate_iterations(ITERATIONS)

Network['Neocortex'](Neurons: tensor(1330)|5 groups, Synapses: tensor(863553)|11 groups){1:TimeResolution(dt=1,)5:SetTarget(target=tensor([1.]),)100:Payoff(initial_payoff=1,)120:Dopamine(tau_dopamine=5,initial_dopamine_concentration=None,)601:Recorder(variables=None,gap_width=0,max_length=None,auto_annotate=True,tag=None,arg_0=['dopamine'],)}
NeuronGroup['NeuronGroup_1', 'NeuronGroup', 'ng'](256){0:NeuronDimension(depth=1,height=16,width=16,input_patterns=None,)220:SimpleDendriteStructure(Proximal_max_delay=1,Distal_max_delay=1,Apical_max_delay=None,proximal_min_delay=0,distal_min_delay=0,apical_min_delay=None,)240:SimpleDendriteComputation(I_tau=None,apical_provocativeness=None,distal_provocativeness=None,)250:RandomInputCurrent(prob_to_spike=0.5,T=10,)380:NeuronAxon(max_delay=1,proximal_min_delay=0,distal_min_delay=0,apical_min_delay=0,have_trace=None,)603:EventRecorder(variables=None,gap_width=0,max_length=None,auto_annotate=True,tag=None,arg_0=['spikes'],)}
NeuronGroup['RefrenceFra

574.0180015563965

# Visualizing

### Finding the refrence Frames

In [263]:
refrences = []

for refrence in L56.neurongroups:
    if 'RefrenceFrame' in refrence.tags:
        refrences.append(refrence)

### Saving the plots

In [265]:
refrence_frame_raster(
    refrence_frame=refrences[1],
    lib = SCREEN_SHOT_PATH,
    last_itr=20,
    step=1
)